In [5]:
import numpy as np
#import biopython as biopy
import pandas as pd

import re

pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
data_path = "../datasets/" # file path to data
analysis_threshold = 10 # number of amino acids either side to analyze
modifications = ["655.3735", "649.3660"] # which modifications we are looking for, as regex strings

Import full `combined_modified_peptide.tsv` file import to `full_peptides` dataframe.

In [7]:
full_peptides = pd.read_csv(data_path + "combined_modified_peptide.tsv", delimiter="\t")
full_peptides

,Peptide Sequence,Modified Sequence,Prev AA,Next AA,Start,End,Peptide Length,Charges,Assigned Modifications,Protein,...,11 Match Type,12 Match Type,2 Match Type,3 Match Type,4 Match Type,5 Match Type,6 Match Type,7 Match Type,8 Match Type,9 Match Type
0,AAALVLQTIWGYK,AAALVLQTIWGYK,R,E,821,833,13,2,NaN,sp|P30999|CTND1_MOUSE,...,unmatched,MS/MS,unmatched,unmatched,MS/MS,MS/MS,MS/MS,unmatched,unmatched,unmatched
1,AAATFNPELITHILDGSPENTRR,AAATFNPELITHILDGSPENTRR,R,R,10,32,23,4,NaN,sp|Q9R0H0|ACOX1_MOUSE,...,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
2,AAAVGIAQVVISR,AAAVGIAQVVISR,R,I,220,232,13,2,NaN,sp|Q925N2|SFXN2_MOUSE,...,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
3,AAAVGIAQVVISRITMAAPGMILLPVIMER,AAAVGIAQVVISRITMAAPGMILLPVIMER,R,L,220,249,30,"3,4",NaN,sp|Q925N2|SFXN2_MOUSE,...,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
4,AAAVGIAQVVISRITMAAPGMILLPVIMERLER,AAAVGIAQVVISRITMAAPGMILLPVIMERLER,R,L,220,252,33,4,NaN,sp|Q925N2|SFXN2_MOUSE,...,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched,MS/MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5447,STFRLALIQLQVSSIKSDNLTR,n[42.0106]STFRLALIQLQVSSIKSDNLTR,M,A,2,23,22,3,N-term(42.0106),sp|Q9JHW2|NIT2_MOUSE,...,MS/MS,unmatched,unmatched,MS/MS,unmatched,unmatched,MS/MS,unmatched,MS/MS,MS/MS
5448,TDAAVSFAK,n[42.0106]TDAAVSFAK,M,D,2,10,9,2,N-term(42.0106),sp|P51881|ADT2_MOUSE,...,unmatched,unmatched,unmatched,MS/MS,unmatched,unmatched,unmatched,unmatched,unmatched,unmatched
5449,TDAAVSFAKDFLAGGVAAAISK,n[42.0106]TDAAVSFAKDFLAGGVAAAISK,M,T,2,23,22,"2,3",N-term(42.0106),sp|P51881|ADT2_MOUSE,...,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS
5450,TDAAVSFAKDFLAGGVAAAISKTAVAPIER,n[42.0106]TDAAVSFAKDFLAGGVAAAISKTAVAPIER,M,V,2,31,30,"2,3,4",N-term(42.0106),sp|P51881|ADT2_MOUSE,...,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS,MS/MS


In [8]:
peptides = full_peptides.iloc[:, np.r_[0, 1, 8]]
peptides

,Peptide Sequence,Modified Sequence,Assigned Modifications
0,AAALVLQTIWGYK,AAALVLQTIWGYK,NaN
1,AAATFNPELITHILDGSPENTRR,AAATFNPELITHILDGSPENTRR,NaN
2,AAAVGIAQVVISR,AAAVGIAQVVISR,NaN
3,AAAVGIAQVVISRITMAAPGMILLPVIMER,AAAVGIAQVVISRITMAAPGMILLPVIMER,NaN
4,AAAVGIAQVVISRITMAAPGMILLPVIMERLER,AAAVGIAQVVISRITMAAPGMILLPVIMERLER,NaN
...,...,...,...
5447,STFRLALIQLQVSSIKSDNLTR,n[42.0106]STFRLALIQLQVSSIKSDNLTR,N-term(42.0106)
5448,TDAAVSFAK,n[42.0106]TDAAVSFAK,N-term(42.0106)
5449,TDAAVSFAKDFLAGGVAAAISK,n[42.0106]TDAAVSFAKDFLAGGVAAAISK,N-term(42.0106)
5450,TDAAVSFAKDFLAGGVAAAISKTAVAPIER,n[42.0106]TDAAVSFAKDFLAGGVAAAISKTAVAPIER,N-term(42.0106)


Format canonicalization of `Assigned Modifications` column

In [9]:
peptides["Assigned Modifications"] = peptides["Assigned Modifications"].str.replace("(", "[", regex=False)
peptides["Assigned Modifications"] = peptides["Assigned Modifications"].str.replace(")", "]", regex=False)
peptides

,Peptide Sequence,Modified Sequence,Assigned Modifications
0,AAALVLQTIWGYK,AAALVLQTIWGYK,NaN
1,AAATFNPELITHILDGSPENTRR,AAATFNPELITHILDGSPENTRR,NaN
2,AAAVGIAQVVISR,AAAVGIAQVVISR,NaN
3,AAAVGIAQVVISRITMAAPGMILLPVIMER,AAAVGIAQVVISRITMAAPGMILLPVIMER,NaN
4,AAAVGIAQVVISRITMAAPGMILLPVIMERLER,AAAVGIAQVVISRITMAAPGMILLPVIMERLER,NaN
...,...,...,...
5447,STFRLALIQLQVSSIKSDNLTR,n[42.0106]STFRLALIQLQVSSIKSDNLTR,N-term[42.0106]
5448,TDAAVSFAK,n[42.0106]TDAAVSFAK,N-term[42.0106]
5449,TDAAVSFAKDFLAGGVAAAISK,n[42.0106]TDAAVSFAKDFLAGGVAAAISK,N-term[42.0106]
5450,TDAAVSFAKDFLAGGVAAAISKTAVAPIER,n[42.0106]TDAAVSFAKDFLAGGVAAAISKTAVAPIER,N-term[42.0106]


In [10]:
def modification_patterns_creation(modifications):

    left_pad = r"(([^\]]*)"
    right_pad = r"([^\[]*))"

    internal = "(M\[("

    for i in range(len(modifications)):
        split_str = modifications[i].split(".")
        whole = split_str[0]
        mantissa = split_str[1]
        
        if i != (len(modifications) - 1):
            internal += r"{}\.{}|".format(whole, mantissa)
        else:
            internal += r"{}\.{}".format(whole, mantissa)
            internal += r"))\]"

    assigned_modification_pattern = internal
    modified_sequence_pattern = left_pad + internal + right_pad
    
    return assigned_modification_pattern, modified_sequence_pattern

assigned_modification_pattern, modified_sequence_pattern = modification_patterns_creation(modifications)
print(assigned_modification_pattern)
print(modified_sequence_pattern)

(M\[(655\.3735|649\.3660))\]
(([^\]]*)(M\[(655\.3735|649\.3660))\]([^\[]*))


In [11]:
methionine_peptides = peptides[peptides["Assigned Modifications"].str.contains(assigned_modification_pattern, na=False)]
methionine_peptides

/var/folders/_5/403kn5_x2kv2k_nh9s2r70340000gn/T/ipykernel_15682/446052648.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  methionine_peptides = peptides[peptides["Assigned Modifications"].str.contains(assigned_modification_pattern, na=False)]


,Peptide Sequence,Modified Sequence,Assigned Modifications
6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,10M[655.3735]
9,AAEMLLFGK,AAEM[649.3660]LLFGK,4M[649.3660]
10,AAEMLLFGK,AAEM[655.3735]LLFGK,4M[655.3735]
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,7M[649.3660]
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,7M[655.3735]
...,...,...,...
5431,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,"N-term[42.0106],2M[655.3735]"
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,"N-term[42.0106],11M[649.3660]"
5442,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[655.3735]KQWK,"N-term[42.0106],11M[655.3735]"
5445,SSMTQNLR,n[42.0106]SSM[649.3660]TQNLR,"N-term[42.0106],3M[649.3660]"


In [12]:
# confirm 1 match per sequence - TODO: ask jayde how to deal with multiple modifications in one sequence
# methionine_peptides = methionine_peptides[methionine_peptides["Modified Sequence"].str.count(modified_sequence_pattern) == 1]

matches_df = methionine_peptides["Modified Sequence"].str.extract(modified_sequence_pattern, expand=False)
methionine_peptides["Analysis Sequence"] = matches_df[0]
methionine_peptides["Analysis Pre-Sequence"] = matches_df[1].str[-analysis_threshold:]  # limit to previous ten amino acids
methionine_peptides["Analysis Post-Sequence"] = matches_df[4].str[:analysis_threshold]  # limit to next ten amino acids

methionine_peptides

,Peptide Sequence,Modified Sequence,Assigned Modifications,Analysis Sequence,Analysis Pre-Sequence,Analysis Post-Sequence
6,AADTIGYPVMIR,AADTIGYPVM[655.3735]IR,10M[655.3735],AADTIGYPVM[655.3735]IR,AADTIGYPV,IR
9,AAEMLLFGK,AAEM[649.3660]LLFGK,4M[649.3660],AAEM[649.3660]LLFGK,AAE,LLFGK
10,AAEMLLFGK,AAEM[655.3735]LLFGK,4M[655.3735],AAEM[655.3735]LLFGK,AAE,LLFGK
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,7M[649.3660],AAESSAM[649.3660]AATEK,AAESSA,AATEK
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,7M[655.3735],AAESSAM[655.3735]AATEK,AAESSA,AATEK
...,...,...,...,...,...,...
5431,MMNGRPGHEPLK,n[42.0106]MM[655.3735]NGRPGHEPLK,"N-term[42.0106],2M[655.3735]",MM[655.3735]NGRPGHEPLK,M,NGRPGHEPLK
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,"N-term[42.0106],11M[649.3660]",SDSRDPASDQM[649.3660]KQWK,SDSRDPASDQ,KQWK
5442,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[655.3735]KQWK,"N-term[42.0106],11M[655.3735]",SDSRDPASDQM[655.3735]KQWK,SDSRDPASDQ,KQWK
5445,SSMTQNLR,n[42.0106]SSM[649.3660]TQNLR,"N-term[42.0106],3M[649.3660]",SSM[649.3660]TQNLR,SS,TQNLR


In [13]:
for i in np.arange(analysis_threshold):
    methionine_peptides[f"Pre-Seq {-10 + i}"] = methionine_peptides["Analysis Pre-Sequence"].str[-10 + i]

for i in np.arange(analysis_threshold):
    methionine_peptides[f"Post-Seq {i + 1}"] = methionine_peptides["Analysis Post-Sequence"].str[i]


display(methionine_peptides.iloc[:, np.r_[1, 3, 4, 6:16]])
display(methionine_peptides.iloc[:, np.r_[1, 3, 5, 16:26]])

,Modified Sequence,Analysis Sequence,Analysis Pre-Sequence,Pre-Seq -10,Pre-Seq -9,Pre-Seq -8,Pre-Seq -7,Pre-Seq -6,Pre-Seq -5,Pre-Seq -4,Pre-Seq -3,Pre-Seq -2,Pre-Seq -1
6,AADTIGYPVM[655.3735]IR,AADTIGYPVM[655.3735]IR,AADTIGYPV,NaN,A,A,D,T,I,G,Y,P,V
9,AAEM[649.3660]LLFGK,AAEM[649.3660]LLFGK,AAE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A,E
10,AAEM[655.3735]LLFGK,AAEM[655.3735]LLFGK,AAE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A,E
11,AAESSAM[649.3660]AATEK,AAESSAM[649.3660]AATEK,AAESSA,NaN,NaN,NaN,NaN,A,A,E,S,S,A
12,AAESSAM[655.3735]AATEK,AAESSAM[655.3735]AATEK,AAESSA,NaN,NaN,NaN,NaN,A,A,E,S,S,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,n[42.0106]MM[655.3735]NGRPGHEPLK,MM[655.3735]NGRPGHEPLK,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
5441,n[42.0106]SDSRDPASDQM[649.3660]KQWK,SDSRDPASDQM[649.3660]KQWK,SDSRDPASDQ,S,D,S,R,D,P,A,S,D,Q
5442,n[42.0106]SDSRDPASDQM[655.3735]KQWK,SDSRDPASDQM[655.3735]KQWK,SDSRDPASDQ,S,D,S,R,D,P,A,S,D,Q
5445,n[42.0106]SSM[649.3660]TQNLR,SSM[649.3660]TQNLR,SS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S


,Modified Sequence,Analysis Sequence,Analysis Post-Sequence,Post-Seq 1,Post-Seq 2,Post-Seq 3,Post-Seq 4,Post-Seq 5,Post-Seq 6,Post-Seq 7,Post-Seq 8,Post-Seq 9,Post-Seq 10
6,AADTIGYPVM[655.3735]IR,AADTIGYPVM[655.3735]IR,IR,I,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AAEM[649.3660]LLFGK,AAEM[649.3660]LLFGK,LLFGK,L,L,F,G,K,NaN,NaN,NaN,NaN,NaN
10,AAEM[655.3735]LLFGK,AAEM[655.3735]LLFGK,LLFGK,L,L,F,G,K,NaN,NaN,NaN,NaN,NaN
11,AAESSAM[649.3660]AATEK,AAESSAM[649.3660]AATEK,AATEK,A,A,T,E,K,NaN,NaN,NaN,NaN,NaN
12,AAESSAM[655.3735]AATEK,AAESSAM[655.3735]AATEK,AATEK,A,A,T,E,K,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,n[42.0106]MM[655.3735]NGRPGHEPLK,MM[655.3735]NGRPGHEPLK,NGRPGHEPLK,N,G,R,P,G,H,E,P,L,K
5441,n[42.0106]SDSRDPASDQM[649.3660]KQWK,SDSRDPASDQM[649.3660]KQWK,KQWK,K,Q,W,K,NaN,NaN,NaN,NaN,NaN,NaN
5442,n[42.0106]SDSRDPASDQM[655.3735]KQWK,SDSRDPASDQM[655.3735]KQWK,KQWK,K,Q,W,K,NaN,NaN,NaN,NaN,NaN,NaN
5445,n[42.0106]SSM[649.3660]TQNLR,SSM[649.3660]TQNLR,TQNLR,T,Q,N,L,R,NaN,NaN,NaN,NaN,NaN


In [14]:
methionine_peptides["Pre-Seq -4"].isna().sum()

330

Temporary Analysis: only selecting sequences that have four on either side

In [15]:
four_analysis = methionine_peptides[(methionine_peptides["Pre-Seq -4"].isna() == False) & (methionine_peptides["Post-Seq 4"].isna() == False)]
four_analysis = four_analysis.iloc[:, np.r_[:6, 12:20]]

four_analysis

,Peptide Sequence,Modified Sequence,Assigned Modifications,Analysis Sequence,Analysis Pre-Sequence,Analysis Post-Sequence,Pre-Seq -4,Pre-Seq -3,Pre-Seq -2,Pre-Seq -1,Post-Seq 1,Post-Seq 2,Post-Seq 3,Post-Seq 4
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,7M[649.3660],AAESSAM[649.3660]AATEK,AAESSA,AATEK,E,S,S,A,A,A,T,E
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,7M[655.3735],AAESSAM[655.3735]AATEK,AAESSA,AATEK,E,S,S,A,A,A,T,E
17,AAHLMDVAGNK,AAHLM[655.3735]DVAGNK,5M[655.3735],AAHLM[655.3735]DVAGNK,AAHL,DVAGNK,A,A,H,L,D,V,A,G
21,AALEMKEEELVSER,AALEM[655.3735]KEEELVSER,5M[655.3735],AALEM[655.3735]KEEELVSER,AALE,KEEELVSER,A,A,L,E,K,E,E,E
27,AALTMFSTIIR,AALTM[649.3660]FSTIIR,5M[649.3660],AALTM[649.3660]FSTIIR,AALT,FSTIIR,A,A,L,T,F,S,T,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5379,AATASPGAGRMDGKPR,n[42.0106]AATASPGAGRM[655.3735]DGKPR,"N-term[42.0106],11M[655.3735]",AATASPGAGRM[655.3735]DGKPR,AATASPGAGR,DGKPR,G,A,G,R,D,G,K,P
5410,ASVLSYESLVHAVAGAVGSVTAMTVFFPLDTAR,n[42.0106]ASVLSYESLVHAVAGAVGSVTAM[649.3660]TVF...,"N-term[42.0106],23M[649.3660]",ASVLSYESLVHAVAGAVGSVTAM[649.3660]TVFFPLDTAR,VAGAVGSVTA,TVFFPLDTAR,S,V,T,A,T,V,F,F
5411,ASVLSYESLVHAVAGAVGSVTAMTVFFPLDTAR,n[42.0106]ASVLSYESLVHAVAGAVGSVTAM[655.3735]TVF...,"N-term[42.0106],23M[655.3735]",ASVLSYESLVHAVAGAVGSVTAM[655.3735]TVFFPLDTAR,VAGAVGSVTA,TVFFPLDTAR,S,V,T,A,T,V,F,F
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,"N-term[42.0106],11M[649.3660]",SDSRDPASDQM[649.3660]KQWK,SDSRDPASDQ,KQWK,A,S,D,Q,K,Q,W,K


In [16]:
four_analysis["Analysis Pre-Sequence"] = four_analysis["Analysis Pre-Sequence"].str[-4:]
four_analysis["Analysis Post-Sequence"] = four_analysis["Analysis Post-Sequence"].str[:4]
four_analysis

,Peptide Sequence,Modified Sequence,Assigned Modifications,Analysis Sequence,Analysis Pre-Sequence,Analysis Post-Sequence,Pre-Seq -4,Pre-Seq -3,Pre-Seq -2,Pre-Seq -1,Post-Seq 1,Post-Seq 2,Post-Seq 3,Post-Seq 4
11,AAESSAMAATEK,AAESSAM[649.3660]AATEK,7M[649.3660],AAESSAM[649.3660]AATEK,ESSA,AATE,E,S,S,A,A,A,T,E
12,AAESSAMAATEK,AAESSAM[655.3735]AATEK,7M[655.3735],AAESSAM[655.3735]AATEK,ESSA,AATE,E,S,S,A,A,A,T,E
17,AAHLMDVAGNK,AAHLM[655.3735]DVAGNK,5M[655.3735],AAHLM[655.3735]DVAGNK,AAHL,DVAG,A,A,H,L,D,V,A,G
21,AALEMKEEELVSER,AALEM[655.3735]KEEELVSER,5M[655.3735],AALEM[655.3735]KEEELVSER,AALE,KEEE,A,A,L,E,K,E,E,E
27,AALTMFSTIIR,AALTM[649.3660]FSTIIR,5M[649.3660],AALTM[649.3660]FSTIIR,AALT,FSTI,A,A,L,T,F,S,T,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5379,AATASPGAGRMDGKPR,n[42.0106]AATASPGAGRM[655.3735]DGKPR,"N-term[42.0106],11M[655.3735]",AATASPGAGRM[655.3735]DGKPR,GAGR,DGKP,G,A,G,R,D,G,K,P
5410,ASVLSYESLVHAVAGAVGSVTAMTVFFPLDTAR,n[42.0106]ASVLSYESLVHAVAGAVGSVTAM[649.3660]TVF...,"N-term[42.0106],23M[649.3660]",ASVLSYESLVHAVAGAVGSVTAM[649.3660]TVFFPLDTAR,SVTA,TVFF,S,V,T,A,T,V,F,F
5411,ASVLSYESLVHAVAGAVGSVTAMTVFFPLDTAR,n[42.0106]ASVLSYESLVHAVAGAVGSVTAM[655.3735]TVF...,"N-term[42.0106],23M[655.3735]",ASVLSYESLVHAVAGAVGSVTAM[655.3735]TVFFPLDTAR,SVTA,TVFF,S,V,T,A,T,V,F,F
5441,SDSRDPASDQMKQWK,n[42.0106]SDSRDPASDQM[649.3660]KQWK,"N-term[42.0106],11M[649.3660]",SDSRDPASDQM[649.3660]KQWK,ASDQ,KQWK,A,S,D,Q,K,Q,W,K


In [17]:
sequences = four_analysis["Analysis Pre-Sequence"].values + four_analysis["Analysis Post-Sequence"].values

In [18]:
for i in sequences:
    print(i)

ESSAAATE
ESSAAATE
AAHLDVAG
AALEKEEE
AALTFSTI
AALTFSTI
GDRTLDSL
GDRTLDSL
VPNDQVGQ
VPNDQVGQ
TAKGLPDP
TAKGLPDP
LRQLDNEV
GEIEELQR
GEIEELQR
SLKDDLID
AGSYPPST
STALHFRI
STALHFRI
QFASTGIP
AIIDYTEG
KIPATIAK
KIPATIAK
RLLLHPNL
VGVSLNVF
VGVSLNVF
DAHAGQQP
HALVTFLF
HALVTFLF
DHYGEDVK
RGNILSQK
PHKQLSER
ASLDFNQK
FDEDDSSL
FDEDDSSL
LNKYESDG
LNKYESDG
DFAHPLLK
RDYLSTHF
VREHEEVR
VREHEEVR
YHLLSVQE
YHLLSVQE
AMKEHFHP
DTQAKEMH
DTQAKEMH
AHVTLSAS
PEEIWFKK
ITEKEETK
ITEKEETK
MLQIEEIK
MLQIEEIK
TEDLAWMR
DVIPNLPH
EELNDLFR
ALSEKTFV
HLFSEGSI
HLFSEGSI
RGLLLDGQ
SAAGAIKN
SAAGAIKN
VNNSLIGQ
VNNSLIGQ
VLRGGGAF
VLRGGGAF
VTLSAYRL
LGAPGSLL
FTDEAHFD
RNDLEYAK
RNDLEYAK
LSAAYPDV
SGLDHTKP
SGLDHTKP
HQGVVGMG
PGLPNNHG
SFGTGVGL
HWPFVVND
HWPFVVND
IAGNGLAM
IAGNGLAM
IAGNGLAM
IAGNGLAM
KLARAILQ
GYDALGDF
INDANMGH
INDANMGH
HALVTFLF
HALVTFLF
LEAAEMIP
LEAAEMIP
VREPQTGI
ITELQVLK
ITELQVLK
HLAGKDSK
HLAGKDSK
YAGQAVLG
YAGQAVLG
VTLSAYRL
KERDDELK
KERDDELK
KEVSDDHK
KEVSDDHK
ALYVRLAL
NGDNLEPS
LNYSFIPT
SQSEMAEI
SQSEMAEI
KNPAYEND
LLATEALN
AILRPHGY
TVLEGAAR
T